In [1]:
import sys, os
from typing import Optional
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/yuyuezhu/Desktop/FRE-GT-9743-Assignment-2
Fixed Income Library is loaded.


# Instructions for HW2:

## 1. Date class and functions


The date folder provides a set of utility functions.

Please familiarize yourself with these date-related functions. You may test and debug them using the code below.

**You don't need to complete any code in this section.**

The test code is written based on APIs derived from the date utilities. When using any functionality from the date module, please call the lower-level utility functions directly, rather than the higher-level APIs.

In [2]:
start_date = "2025-05-25"
term = "3M"
business_day_convention = "F"
holiday_convention = "USGS"
accrual_basis = 'ACT/ACT'
test_date = '2025-12-21' # you can change this date to test other dates

In [3]:
end_date = qfAddPeriod(start_date, term, business_day_convention, holiday_convention)
qfAccrued(start_date, end_date, accrual_basis, business_day_convention, holiday_convention)

0.2520547945205479

For example, 'add_period' is the lower-level of 'qdAddPeriod'.

In [4]:
end_date = add_period(
    Date(start_date),
    Period(term),
    BusinessDayConvention(business_day_convention),
    HolidayConvention(holiday_convention),
)
qfAccrued(start_date, end_date, accrual_basis, business_day_convention, holiday_convention)

0.2520547945205479

In [5]:
qfMoveToBusinessDay(test_date, business_day_convention, holiday_convention)

'2025-12-22'

In [6]:
qfIsBusinessDay(test_date, holiday_convention)

False

In [7]:
test_date = '2026-01-01'
qfIsHoliday(test_date, holiday_convention)

True

In [8]:
test_date = '2025-12-21'
print(qfIsEndOfMonth(test_date, holiday_convention))
test_date = '2025-12-31'
print(qfIsEndOfMonth(test_date, holiday_convention))

False
True


In [9]:
start_date = "2025-05-25"
end_date = "2027-01-30"
period = "6M"
business_day_convention = "F"
holiday_convention = "USGS"
accrual_basis = "ACT/360"
rule = "BACKWARD"
end_of_month = False
fix_in_arrear = True
fixing_offset = "1D"
payment_offset = "2D"
pay_business_day_convention = "F"
pay_holiday_convention = "USGS"
qfCreateSchedule(
    start_date,
    end_date,
    period,
    holiday_convention,
    business_day_convention,
    accrual_basis,
    rule,
    end_of_month,
    fix_in_arrear,
    fixing_offset,
    payment_offset,
    pay_business_day_convention,
    pay_holiday_convention,
)

,StartDate,EndDate,FixingDate,PaymentDate,Accrued
0,"May 27th, 2025","July 30th, 2025","July 31st, 2025","August 1st, 2025",0.177778
1,"July 30th, 2025","January 30th, 2026","February 2nd, 2026","February 3rd, 2026",0.511111
2,"January 30th, 2026","July 30th, 2026","July 31st, 2026","August 3rd, 2026",0.502778
3,"July 30th, 2026","February 1st, 2027","February 2nd, 2027","February 3rd, 2027",0.516667


## 2. Build Product

The product specifications and code templates for Bullet Cashflow and Overnight Index will be provided.

You are required to complete the implementations for Interest Rate Stream and Swap. After you complete the code, you'll be able to display the full information of the product.

If you are not familiar with these two products, please review the relevant past lectures on Notion before starting.


1. Bullet Cashflow

- Principal leg is a single payment

In [10]:
# Product Bullet Cashflow

termination_date = '2026-05-26'
currency = 'USD'
notional = 1e6
long_or_short = 'LONG'
payment_date = qfAddPeriod(termination_date, '2D', 'F', 'USGS')

# qfCreateProductBulletCashflow is one of api, check lower-level class 'ProductBulletCashflow' for implementation

prod_bullet_cashflow = qfCreateProductBulletCashflow(
    termination_date,
    currency,
    notional,
    long_or_short,
    payment_date
)

Display the product information

In [11]:
display_info = [
    ["Product Type", prod_bullet_cashflow.product_type],
    ["Notional", prod_bullet_cashflow.notional],
    ["Currency", prod_bullet_cashflow.currency.value_str],
    ["Long Or Short", prod_bullet_cashflow.long_or_short.to_string().upper()],
]

display_info.append(["Termination Date", prod_bullet_cashflow.termination_date.ISO()])
display_info.append(["Payment Date", prod_bullet_cashflow.payment_date.ISO()])
display_info = pd.DataFrame(display_info, columns=["Name", "Value"])

In [12]:
display_info

,Name,Value
0,Product Type,PRODUCT_BULLET_CASHFLOW
1,Notional,1000000.0
2,Currency,USD
3,Long Or Short,LONG
4,Termination Date,2026-05-26
5,Payment Date,2026-05-28


2. Overnight Index Cashflow

- interest is calculated by compounding daily overnight index rates over an accrual period, with the resulting interest (plus any spread) paid as a single cashflow on the payment date.

In [13]:
# parametric product
effecitve_date = '2026-03-26'
termination_date = '2027-03-26'
term = '1Y'
overnight_index = 'SOFR-1B'
notional = 1e6
compounding_method = 'compound'
spread = 0.005
business_day_convention = 'F'
holiday_convention = 'USGS'
pay_offset = '2D'
payment_date = qfAddPeriod(termination_date, pay_offset, business_day_convention, holiday_convention)

product_on_cashflow = qfCreateProductOvernightIndexCashflow(
    effecitve_date,
    term,
    overnight_index,
    notional,
    compounding_method,
    spread,
    payment_date)

In [14]:
display_info = [
    ["Product Type", product_on_cashflow.product_type],
    ["Notional", product_on_cashflow.notional],
    ["Currency", product_on_cashflow.currency.value_str],
    ["Long Or Short", product_on_cashflow.long_or_short.to_string().upper()],
]        
display_info.append(['Effective Date', product_on_cashflow.effective_date.ISO()])
display_info.append(['Termination Date', product_on_cashflow.termination_date.ISO()])
display_info.append(['ON Index', product_on_cashflow.on_index.name()])
display_info.append(['Compounding Method', product_on_cashflow.compounding_method.to_string().upper()])
display_info.append(['Spread', product_on_cashflow.spread])
display_info.append(['Payment Date', product_on_cashflow.payment_date.ISO()])

display_info = pd.DataFrame(display_info, columns=["Name", "Value"])

In [15]:
display_info

,Name,Value
0,Product Type,PRODUCT_OVERNIGHT_INDEX_CASHFLOW
1,Notional,1000000.0
2,Currency,USD
3,Long Or Short,LONG
4,Effective Date,2026-03-26
5,Termination Date,2027-03-29
6,ON Index,SOFRON Actual/360
7,Compounding Method,COMPOUND
8,Spread,0.005
9,Payment Date,2027-03-30


TODO:

Complete the code in linearproduct.py so that your results match the expected output shown below.

Hint: You may think of an Interest Rate Stream as a combination of an Overnight Index Cashflow and a Fixed Accrued component. 

A Swap can then be viewed as two Interest Rate Streams with opposite directions.

In [16]:
effective_date = '2025-05-25'
termination_date = '2027-05-25'
term = '2Y'
pay_offset = '2D'
on_index = 'SOFR-1B'
fixed_rate = 0.04
pay_or_rec = 'pay'
notional = 1e6
accrual_peroid = '1Y'
accrual_basis = 'ACT/360'
floating_leg_accrual_period = '1Y'
business_day_convention = 'F'
holiday_convention = 'USGS'
spread = 0.005
compounding_method = 'compound'

product_on_rfr_swap = qfCreateProductRFRSwap(
    effective_date,
    term,
    pay_offset,
    on_index,
    fixed_rate,
    pay_or_rec,
    notional,
    accrual_peroid,
    accrual_basis,
    floating_leg_accrual_period,
    business_day_convention,
    holiday_convention,
    spread,
    compounding_method)

ValueError: too many values to unpack (expected 2)

In [ ]:
display_info = [
    ["Product Type", product_on_rfr_swap.product_type],
    ["Notional", product_on_rfr_swap.notional],
    ["Currency", product_on_rfr_swap.currency.value_str],
    ["Long Or Short", product_on_rfr_swap.long_or_short.to_string().upper()],
]      

display_info.append(['Effective Date', product_on_rfr_swap.effective_date.ISO()])
display_info.append(['Termination Date', product_on_rfr_swap.termination_date.ISO()])
display_info.append(['Payment Offset', product_on_rfr_swap.pay_offset.__str__()])
display_info.append(['ON Index', product_on_rfr_swap.on_index.name()])
display_info.append(['Fixed Rate', product_on_rfr_swap.fixed_rate])
display_info.append(['Pay Or Receive', product_on_rfr_swap.pay_or_rec.to_string().upper()])
display_info.append(['Accrual Period', product_on_rfr_swap.accrual_period.__str__()])
display_info.append(['Accrual Basis', product_on_rfr_swap.accrual_basis.value_str])
display_info.append(['Floating Leg Accrual Period', product_on_rfr_swap.floating_leg_accrual_period.__str__()])
display_info.append(['Business Day Convention', product_on_rfr_swap.pay_business_day_convention.value_str])
display_info.append(['Holiday Convention', product_on_rfr_swap.pay_holiday_convention.value_str])

display_info = pd.DataFrame(display_info, columns=["Name", "Value"])

display_info

,Name,Value
0,Product Type,PRODUCT_RFR_SWAP
1,Notional,1000000.0
2,Currency,USD
3,Long Or Short,LONG
4,Effective Date,2025-05-25
5,Termination Date,2027-05-25
6,Payment Offset,2D
7,ON Index,SOFRON Actual/360
8,Fixed Rate,0.04
9,Pay Or Receive,PAY


In [ ]:
product_on_rfr_swap.floating_leg_cash_flow(1).notional

AttributeError: 'ProductRFRSwap' object has no attribute 'floating_leg_'

## 3. Finalize Registry (Singleton Example)

In this section, please check the OptionStrategyRegistry folder.

You are required to complete a class named OptionStrategyRegistry, which provides the basic functionality for registering option strategies. The class should first load the file strategies.yaml to initialize a default dictionary of strategies. In addition, users should be able to add new strategies via the register method, which must accept both list and dict inputs.

Please use the OptionStrategy class defined above as the fundamental building block for each strategy.

Your implementation should be able to run successfully with the following code segments.

FYI, the same singleton techniques could be checked in the Data Conventions, Fixings and Indices in FixedIncomeLib as well.

In [ ]:
import yaml
import sys, os
import numpy as np
import pandas as pd
from scipy.stats import norm
from typing import Tuple, Optional
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
sys.path.append(os.path.dirname(os.getcwd()))
from OptionStrategyRegistry import *

KeyboardInterrupt: 

In [ ]:
# instantiate strategy registry
print(OptionStrategyRegistry().list_registry_keys())
# insert repeated registry
repeated_strategy = {
    'OPT_TYPE' : ['C', 'P'],
    'DELTA_STRIKE' : [0.5, 0.5],
    'WEIGHT' : [1, 1]}
OptionStrategyRegistry().register('ATM_STRADDLE', repeated_strategy)
print(OptionStrategyRegistry().list_registry_keys())

['ATM_STRADDLE', 'DELTA25_RR', 'DELTA25_STRANGLE']
['ATM_STRADDLE', 'DELTA25_RR', 'DELTA25_STRANGLE']


In [ ]:
# register a new one on the fly (via dict)
new_strategy = {
    'OPT_TYPE' : ['C', 'P'],
    'DELTA_STRIKE' : [0.25, 0.25],
    'WEIGHT' : [1, 1]}
OptionStrategyRegistry().register('DELTA_25_STRADDLE', repeated_strategy)
print(OptionStrategyRegistry().list_registry_keys())

['ATM_STRADDLE', 'DELTA25_RR', 'DELTA25_STRANGLE', 'DELTA_25_STRADDLE']
